In [44]:
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoTokenizer, DataCollatorForSeq2Seq

In [45]:
dataset = "billsum"
seed_num = 1
model_name = "google-t5/t5-small"

In [46]:
loaded_dataset = load_dataset(dataset, split="ca_test")
# loaded_dataset = loaded_dataset.train_test_split(test_size=0.2, seed=seed_num, shuffle=True)

In [62]:
loaded_dataset
# make the dataset into a pandas dataframe
df = pd.DataFrame(loaded_dataset['train'])
# add the test dataset to the dataframe
df = pd.concat([df, pd.DataFrame(loaded_dataset['test'])], ignore_index=True)

In [63]:
df

,text,summary,title
0,The people of the State of California do enact...,Existing law establishes the State Coastal Con...,An act to add Chapter 10 (commencing with Sect...
1,The people of the State of California do enact...,"Existing law, the Safe Neighborhoods and Schoo...",An act to add Article 10 (commencing with Sect...
2,The people of the State of California do enact...,The California Revised Uniform Limited Liabili...,An act to amend Section 17701.02 of the Corpor...
3,The people of the State of California do enact...,"Under existing law, the Public Utilities Commi...",An act to amend Section 399.16 of the Public U...
4,The people of the State of California do enact...,(1) Existing law authorizes a county board of ...,"An act to amend Sections 44332, 44332.5, and 4..."
...,...,...,...
1232,The people of the State of California do enact...,Existing law provides for the annual assessmen...,"An act to amend and renumber Section 674 of, a..."
1233,The people of the State of California do enact...,Existing law generally sets forth the requirem...,An act to add Chapter 6.7 (commencing with Sec...
1234,The people of the State of California do enact...,Existing law makes it a misdemeanor to place o...,An act to\nadd\namend\nSection\n3055 to\n556.4...
1235,The people of the State of California do enact...,Existing law prohibits a person from being tri...,An act to add Chapter 16 (commencing with Sect...


In [64]:
# Tokenize the summary column
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [65]:
prefix = "summarize: "  # Required so the T5 model knows that we are going to summarize
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(text_target=examples["summary"])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)
tokenized_dataset = loaded_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1772 > 512). Running this sequence through the model will result in indexing errors
Map: 100%|██████████| 989/989 [00:01<00:00, 530.77 examples/s]
